In [1]:
from BoW import getCountVector
import pandas as pd
from torch import nn
import torch
from ffnn import FFNN
import numpy as np
from scipy.sparse import coo_matrix
from Preprocessing import getVectorLabels

C:\Users\jespe\AppData\Local\Temp\ipykernel_13640\3382932014.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("Archive/arxiv_train.csv", nrows=100)
vectoriser = getCountVector(df)
labels = getVectorLabels(df)

In [3]:
coo = coo_matrix(vectoriser)

values = coo.data
indices = np.vstack((coo.row, coo.col))

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = coo.shape

tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))

C:\Users\jespe\AppData\Local\Temp\ipykernel_13640\857494844.py:10: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:620.)
  tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))


In [9]:
model = FFNN(size_vocabulary=tensor.shape[1])

output = model(tensor[0])
output

C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([0.1359, 0.0718, 0.1026, 0.0720, 0.1148, 0.1222, 0.1344, 0.0966, 0.0661,
        0.0836], grad_fn=<SoftmaxBackward0>)

In [5]:
num_epochs = 50

optimiser = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    optimiser.zero_grad()
    # Forward pass: Compute predicted y by passing x to the model
    predicted_labels = model(tensor)

    # Compute and print loss
    loss = loss_function(predicted_labels, labels)
    if epoch % 10 == 0:  # Print every 10 epochs
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    loss.backward()
    optimiser.step()

C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch 0, Loss: 2.299877643585205
Epoch 10, Loss: 2.2816240787506104
Epoch 20, Loss: 2.2484796047210693
Epoch 30, Loss: 2.2048418521881104
Epoch 40, Loss: 2.151268243789673


In [8]:
# Set the model to evaluation mode - important for batch normalization and dropout layers
# Unnecessary in this situation but added for best practices
model.eval()
#size = len(dataloader.dataset)
#num_batches = len(dataloader)
test_loss, correct = 0, 0
test_data = pd.read_csv("Archive/arxiv_test.csv", nrows=100)
test_data["label"] = getVectorLabels(test_data)

# Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
# also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
with torch.no_grad():
    for (_, row) in test_data.iterrows():
        abstract = row["abstract"]
        label = row["label"]

        pred = model(abstract)
        test_loss += loss_function(pred, label).item()
        correct += (pred.argmax(1) == label).type(torch.float).sum().item()

test_loss /= 100
correct /= 100
print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

TypeError: linear(): argument 'input' (position 1) must be Tensor, not str